#### Package Import

In [1]:
import numpy as np
import pandas as pd
import FdsPy.pa.mypaengine as pa
import FdsPy.qe.myqengine as qe
import warnings
warnings.filterwarnings("ignore")

### 1. Screen for Top Holder Data using FQL Arrays

In [4]:
#set universe to NVDA-US
univ = qe.IdUniverse(ids = ['NVDA-US'],
                    universe_type ='Equity')

#set to latest month end
time_series = qe.TimeSeries(start_date='-11M',
                            end_date = '0M',frequency='M')

#Define FQL Formulas
formulas = {'hldr_id':'OS_TOP_HLDR_ID(ALL,#DATE,,MTD,,A,SEC)',
            'hldr_name':'OS_TOP_HLDR_NAME(ALL,#DATE,,MTD,,A,SEC,"EN")',
            'shares':'OS_TOP_HLDR_POS(ALL,#DATE,,MTD,,A,SEC)',
            'mv':'OS_TOP_HLDR_MV(ALL,#DATE,,MTD,,A,SEC,USD)',
            'hldr_type':'OS_TOP_HLDR_INVSTR(ALL,#DATE,,MTD,,A,SEC)',
            'aum':'OS_TOP_HLDR_AUM(ALL,#DATE,,MTD,,A,SEC,USD)'
            }

#Calculate
q_req = qe.Calculation(universe=univ, dates = time_series, data_dict=formulas,is_array=True,source= 'FqlExpression').query()


### 2. Explode Arrays & Clean Data

In [5]:
df_temp = q_req.data.copy()
df_temp = df_temp.set_index(['DATE'])
                    

#Expand arrays and rejoin
df = pd.concat([df_temp['hldr_id'].explode() ,df_temp['hldr_name'].explode(),df_temp['shares'].explode(),df_temp['mv'].explode(),df_temp['hldr_type'].explode(),df_temp['aum'].explode()],axis=1)
df=df.reset_index().set_index(['DATE','hldr_id'])
df.head()

hldr_name       shares  \
DATE     hldr_id                                                            
20220930 F72998                     The Vanguard Group, Inc.  193529915.0   
             M4004630        Vanguard Total Stock Market ETF   70915509.0   
             M4004543                Vanguard 500 Index Fund   56739282.0   
             M4004567             Vanguard Growth Index Fund   19568794.0   
             M4004569      Vanguard Institutional Index Fund   18164860.0   

                                       mv             hldr_type        aum  
DATE     hldr_id                                                            
20220930 F72998        23492596381.849998   Mutual Fund Manager  6649219.0  
             M4004630       8608433637.51  Exchange Traded Fund        NaN  
             M4004543       6887581441.98  Exchange Traded Fund        NaN  
             M4004567       2375455903.66  Exchange Traded Fund        NaN  
             M4004569        2205032355.4         Open-End Fund        NaN

In [6]:
#archive
#df.to_pickle('../../data/Nvidia Holder Analysis/all_holders_last_12m.pkl')
#read archived results
#df=pd.read_pickle('../../data/Nvidia Holder Analysis/all_holders_last_12m.pkl')